In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pkgutil, importlib

In [0]:
import sys
sys.path.append("/Workspace/Users/mayur10594@gmail.com/ETL_project")
from config.utils import get_logger
logger = get_logger("dataloadinitiate")

In [0]:
import json, yaml, os
from config.utils import load_config,create_tables,load_all_schemas


In [0]:
with open("../config/pipeline.yaml", "r") as f:
    pipeline_config = yaml.safe_load(f)

In [0]:
from silver_transformations.add_audit_cols import add_audit_cols

In [0]:
schemas_map = load_all_schemas()
print("All schemas loaded:", schemas_map.keys())

In [0]:
dbutils.widgets.text("fileList", "")
dbutils.widgets.text("taxYear", "")
dbutils.widgets.text("clientId", "")
dbutils.widgets.text("env","")
dbutils.widgets.text("bronze_path", "")
dbutils.widgets.text("maps","")
fileList = dbutils.widgets.get("fileList")
taxYear = dbutils.widgets.get("taxYear")
clientId = dbutils.widgets.get("clientId")
env = dbutils.widgets.get("env")
bronze_path = dbutils.widgets.get("bronze_path")
maps=json.loads(dbutils.widgets.get("maps"))
logger.info(f"fileList: {fileList}, taxYear: {taxYear}, clientId: {clientId}, env: {env}, bronze_path: {bronze_path}")

In [0]:
# if "," in fileList:
#     files=list(fileList.split(","))
#     logger.info(f"files to be processed: {fileList}")
# else:
#     files=[fileList]
#     logger.info(f"only single file to be processed: {fileList}")

In [0]:
# maps=[]
# for file in files:
#     maps.append(file.split("_")[0])
#     #print(file)
# logger.info(f"maps to be processed are {maps}")

In [0]:
path="/Workspace/Users/mayur10594@gmail.com/ETL_project/config/etl_main.yaml"
env_config,tables_config=load_config(path,env)

In [0]:
# in_path=env_config['incoming_path']

In [0]:
# in_files=[f.name for f in dbutils.fs.ls(in_path)]
# for file in files:
#         if file in in_files:
#             #print(f"File {file} exists")
#             logger.info(f"File {file} exists")
#         else:
#             #print(f"File {file} does not exist in incoming folder, Please check & re-run again")
#             logger.error(f"File {file} does not exist in incoming folder, Please check & re-run again")
#             dbutils.notebook.exit("Missing File")
        

In [0]:
# def add_audit_cols(df):
#     df = df.withColumn("create_date",lit(current_date())).withColumn("create_user",lit('cpprod'))
#     return df

In [0]:
# Dictionary to hold DataFrames
dfs = {}

for map_name in maps:
    # Build CSV path
    # csv_path = f"{in_path}/{map_name}_data.csv"
    parquet_path=f"{bronze_path}/{map_name}_data/"
    # Evaluate schema string (assumes map_name_schema exists as a variable)
    #schema = schemas_map[f"{map_name}.{map_name}"]
    # Read CSV into DataFrame
    transformations = pipeline_config[map_name]
    df = spark.read.parquet(parquet_path)
    #df = spark.read.csv(csv_path, schema=schema, header=True)
    df = df.transform(add_audit_cols(df,f"{map_name}_data"))
    # Store in dictionary
    dfs[map_name] = df
    #print(f"df_{map_name} is created and stored in dfs['{map_name}']")
    logger.info(f"df_{map_name} is created and stored in dfs['{map_name}']")


In [0]:
dfs['cards'].display()

In [0]:
for name,df in dfs.items():
    count={}
    df.write.format("delta").options(mergeSchema=True).mode('append').saveAsTable(f"cp_database.{clientId}_{name}")
    count[name]=df.count()
    #print(f"{df} written to delta table cp_database.{clientId}_{name}")
    logger.info(f"{df} written to delta table cp_database.{clientId}_{name}")


In [0]:
logger.info(f"exiting the dataloadinitiate notebook")
dbutils.notebook.exit(json.dumps({
    "status": "OK",
    "message": "All files processed successfully",
    "counts": count
}))